Biopython trial

In [1]:
from Bio import Entrez
import pandas as pd
import re
import seaborn as sns

sns.set_style("whitegrid")
sns.set_context("poster")

#replace with whoever is using this
Entrez.email = 'masashi.omori@gmail.com'

In [13]:
#Find how many articles on asthma we can find on PubMed
# handle = Entrez.egquery(term='asthma')
# record = Entrez.read(handle)
# for row in record['eGQueryResult']:
#     if row['DbName']=='pubmed':
#         print(row['Count'])
def get_citations_for_pmid(pmid):
    results = Entrez.read(Entrez.elink(dbfrom="pubmed", db="pmc",
                                       LinkName="pubmed_pmc_refs", id=pmid))
    try:
        pmc_ids = [link["Id"] for link in results[0]["LinkSetDb"][0]["Link"]]
        return pmc_ids
    except:
        return 0 

In [3]:
from Bio import Entrez
pmid = "30006538"
results = Entrez.read(Entrez.elink(dbfrom="pubmed", db="pmc",
                                   LinkName="pubmed_pmc_refs", id=pmid))
try:
    pmc_ids = [link["Id"] for link in results[0]["LinkSetDb"][0]["Link"]]
    len(pmc_ids)
except:
    print 'no citations'


no citations


In [4]:
#use Bio.Entrez.efetch function to donwload PubMed IDs of these articles
from Bio import Entrez
#handle = Entrez.esearch(db='pubmed', term='asthma',retmax=100)
handle = Entrez.esearch(db='pubmed', term='asthma',retmax=100000)

record = Entrez.read(handle)
handle.close()
idlist = record["IdList"]
idlist[:2], len(idlist)


(['30025179', '30024833'], 100000)

# grab citations


In [21]:
citations = []
print len(idlist)
#for pmid in idlist[len(idlist)-10:]:
for pmid in idlist:
    cit =  get_citations_for_pmid(pmid)
    
    citations.append({pmid: cit})

100000


URLError: <urlopen error [SSL: UNKNOWN_PROTOCOL] unknown protocol (_ssl.c:661)>

In [ ]:
for i in range(len(idlist)-1, -1,-1):
    try:
        cit = get_citations_for_pmid(idlist[i])
        citations.append({pmid: cit})
    except Exception as e:
        print 'failed index {}'.format(i)
        print e

In [ ]:
import numpy as np
np.unique(citations)

In [24]:
df = pd.DataFrame(citations, columns=['pmid','referenced_by'])

In [28]:
citations

[{'30025179': 0},
 {'30024833': 0},
 {'30024788': 0},
 {'30024335': 0},
 {'30024116': 0},
 {'30022785': 0},
 {'30022782': 0},
 {'30022556': 0},
 {'30022479': 0},
 {'30022403': 0},
 {'30022363': 0},
 {'30022059': 0},
 {'30021914': 0},
 {'30021811': 0},
 {'30021810': 0},
 {'30021797': 0},
 {'30021766': 0},
 {'30021597': 0},
 {'30021582': 0},
 {'30021546': 0},
 {'30021447': 0},
 {'30021378': 0},
 {'30020687': 0},
 {'30020668': 0},
 {'30020660': 0},
 {'30021208': 0},
 {'30021120': 0},
 {'30021119': 0},
 {'30020806': 0},
 {'30020048': 0},
 {'30020041': 0},
 {'30019780': 0},
 {'30019775': 0},
 {'30019762': 0},
 {'30019677': 0},
 {'30019675': 0},
 {'30019503': 0},
 {'30019070': 0},
 {'30018955': 0},
 {'30018765': 0},
 {'30018677': 0},
 {'30017826': 0},
 {'30017752': 0},
 {'30017554': 0},
 {'30017541': 0},
 {'30017262': 0},
 {'30017214': 0},
 {'30016982': 0},
 {'30016877': 0},
 {'30016387': 0},
 {'30016216': 0},
 {'30016172': 0},
 {'30016129': 0},
 {'30016124': 0},
 {'30016026': 0},
 {'3001582

In [7]:
#Use medline records in medline flat-file format and use Bio.Medline to parse them
from Bio import Medline
handle = Entrez.efetch(db='pubmed', id=idlist, rettype='medline', retmode='text')
records = Medline.parse(handle)

In [8]:
#records is an ITERATOR, ie you can only iterate them once. Convert to list if you want to save them
records = list(records)

In [9]:
len(records)

10000

In [7]:
# #print out some information
# print 'title: {}'.format(records[1].get("TI","?"))
# print 'authors: {}'.format(records[1].get("AU","?"))
# print "source: {}".format(records[1].get("SO","?"))

In [8]:
tuples = []
for record in records:
    try:
        title = record['TI']
        authors = record['AU']
        crdt = record['CRDT'][0][:10]
    except Exception as e:
        continue
    tuples.append((title, authors, crdt))
pubmed_df = pd.DataFrame(tuples, columns=['title','authors','crdt'])

In [9]:
pubmed_df.head(2)

,title,authors,crdt
0,Alcohol-induced respiratory symptoms improve a...,"[Glicksman JT, Parasher AK, Doghramji L, Braue...",2018/07/15
1,Functional and morphological differences of th...,"[Michaudel C, Fauconnier L, Jule Y, Ryffel B]",2018/07/15


In [10]:
pubmed_df_counts = pubmed_df.groupby('crdt').size()    
pubmed_df_counts.plot(figsize=(20,5))

In [11]:
#some parser functions for the abstract of pubmed articles
def get_methods(record):
    '''returns the METHODS: section of the record'''
    beginning_index = record['AB'].find("METHODS:")+len("METHODS:")
    ending_index = get_next_section_index(record['AB'][beginning_index:])
    return record['AB'][beginning_index:beginning_index + ending_index].strip()

def get_conclusion(record):
    '''returns CONCLUSION section of record'''
    #beginning_index = record['AB'].find("CONCLUSION:")+len("CONCLUSION:")
    conclusion_names = ["CONCLUSIONS:","CONCLUSION:", "Conclusion:"]
    beginning_index = max([re.search(i,record['AB'],re.IGNORECASE).end() \
                        for i in conclusion_names if re.search(i,record['AB'],re.IGNORECASE)])
    #print beginning_index
#     beginning_index = max([record['AB'].find("CONCLUSIONS:")+len("CONCLUSIONS:"),\
#                           record['AB'].find("CONCLUSION:")+len("CONCLUSION:")])
    ending_index = get_next_section_index(record['AB'][beginning_index:])
    return record['AB'][beginning_index:beginning_index + ending_index].strip()


def get_next_section_index(record_string):
    '''returns the next section index. Used to limit output of other get_section functions'''
    section_names = ["RATIONALE:","OBJECTIVES:","METHODS:","RESULTS:","CONCLUSION:",\
                    "OBJECTIVE:","METHOD:","RESULT:"]
    try:
        #ending_index = min([record_string.find(i) for i in section_names if record_string.find(i) > 0])
        ending_index = min([re.search(i,ss,re.IGNORECASE).start() for i in section_names if re.search(i,ss,re.IGNORECASE)])
        return ending_index
    except:
        #no more sections. return large number
        return 9999999

# def get_next_section_index2(record_string):
#     '''returns the next section index. Used to limit output of other get_section functions'''
#     section_names = ["RATIONALE:","OBJECTIVES:","METHODS:","RESULTS:","CONCLUSION:",\
#                     "OBJECTIVE:","METHOD:","RESULT:","BACKGROUND:"]
    
#     try:
#         ending_index = min()
#         ending_index = min([record_string.find(i) for i in section_names if record_string.find(i) > 0])
#         return ending_index
#     except:
#         #no more sections. return large number
#         return 9999999


In [14]:
records[len(records)-1]

{'AB': 'To advance the development of bronchodilators for asthma and chronic obstructive pulmonary disease (COPD), this study was designed to investigate the mechanism of functional antagonism between &beta;(2)-adrenergic and muscarinic M(2) receptors, focusing on allosteric effects and G proteins/ion channels coupling. Muscarinic receptor antagonists (tiotropium, glycopyrronium, atropine) synergistically enhanced the relaxant effects of &beta;(2)-adrenergic receptor agonists (procaterol, salbutamol, formoterol) in guinea pig trachealis. This crosstalk was inhibited by iberitoxin, a large-conductance Ca(2+)-activated K(+) (KCa) channel inhibitor, whereas it was increased by verapamil, a L-type voltage-dependent Ca(2+) (VDC) channel inhibitor; additionally, it was enhanced after tissues were incubated with pertussis or cholera toxin. This synergism converges in the G proteins (Gi, Gs)/KCa channel/VDC channel linkages. Muscarinic receptor antagonists competitively suppressed, whereas, &b

In [12]:
from gensim.summarization.summarizer import summarize

print records[len(records)-1]['AB']
print '\n'
print summarize(records[len(records)-1]['AB'])

To advance the development of bronchodilators for asthma and chronic obstructive pulmonary disease (COPD), this study was designed to investigate the mechanism of functional antagonism between &beta;(2)-adrenergic and muscarinic M(2) receptors, focusing on allosteric effects and G proteins/ion channels coupling. Muscarinic receptor antagonists (tiotropium, glycopyrronium, atropine) synergistically enhanced the relaxant effects of &beta;(2)-adrenergic receptor agonists (procaterol, salbutamol, formoterol) in guinea pig trachealis. This crosstalk was inhibited by iberitoxin, a large-conductance Ca(2+)-activated K(+) (KCa) channel inhibitor, whereas it was increased by verapamil, a L-type voltage-dependent Ca(2+) (VDC) channel inhibitor; additionally, it was enhanced after tissues were incubated with pertussis or cholera toxin. This synergism converges in the G proteins (Gi, Gs)/KCa channel/VDC channel linkages. Muscarinic receptor antagonists competitively suppressed, whereas, &beta;(2)-

In [13]:
ss = get_conclusion(records[len(records)-1])
# section_names = ["RATIONALE:","OBJECTIVES:","METHODS:","RESULTS:","CONCLUSION:",\
#                     "OBJECTIVE:","METHOD:","RESULT:","BACKGROUND:"]
# re.search("CONCLUSION:",ss,re.IGNORECASE).start()

# [re.search(i,ss,re.IGNORECASE).start() for i in section_names if re.search(i,ss,re.IGNORECASE)]
ss

ValueError: max() arg is an empty sequence

In [ ]:
re.search("Conclusion:", records[9]['AB'], re.IGNORECASE)

In [ ]:
for i, rec in enumerate(records):
    try:
        print i, get_conclusion(rec)
    except:
        print i
        continue

In [ ]:
get_methods(records[1])

In [ ]:
records[1]['AB']

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
sid = SentimentIntensityAnalyzer()
sid.polarity_scores(get_conclusion(records[1]))

In [ ]:
tuples = []
for rec in records:
    try:
        conclusion = get_conclusion(rec)
    except:
        continue
    scores = sid.polarity_scores(conclusion)
    tuples.append((conclusion, scores, scores['compound']))

In [ ]:
df_scores = pd.DataFrame(tuples, columns=['conclusion','scores','compound'])

In [ ]:
df_scores.to_csv('scores.csv')

In [ ]:
records[1]